In [2]:
import pandas as pd
import numpy as np

In [3]:
#Load the Data
train = pd.read_csv('../Input/train.csv')
test = pd.read_csv('../Input/test.csv')
features = [c for c in train.columns if c not in ['ID_code', 'target']]
target = train['target']

In [5]:
from sklearn.model_selection import StratifiedKFold

In [49]:
X_train = train[features].values
X_test = test[features].values
y_train = target.values

In [50]:
folds = 4
skf = StratifiedKFold(n_splits=folds, random_state=13)

In [51]:
oof_train = np.zeros(X_train.shape[0])
oof_test = np.zeros(X_test.shape[0])

In [52]:
X_test.shape

(200000, 200)

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [55]:
def train_RF(X,y):
    model = RandomForestClassifier(n_jobs=-1)
    model.fit(X,y)
    return model

def predict_RF(model, X): # Для бинарной классификации
    return model.predict_proba(X)[:,1]

In [56]:
splits = skf.split(X_train, y_train)
for i in enumerate(splits):
    m = train_RF(X_train[i[1][0],:], y_train[i[1][0]])
    pred = predict_RF(m, X_train[i[1][1],:])
    true = y_train[i[1][1]]
    oof_train[i[1][1]] = pred
    print(i[0], roc_auc_score(true, pred))
    oof_test += predict_RF(m, X_test)/folds

0 0.697356431998669
1 0.6952395926804965
2 0.6978560452905542
3 0.7025973515010109


In [44]:
oof_train

array([0. , 0.2, 0. , ..., 0. , 0.2, 0.1])

In [46]:
roc_auc_score(y_train, oof_train)

0.6987326678877949

In [45]:
oof_test

array([0.1  , 0.1  , 0.175, ..., 0.05 , 0.075, 0.025])